In [1]:
import time
import matplotlib.pyplot as plt
import torch
import numpy as np
from laplacian_py import laplacian_solver

res = 50
max_val = 80
cost_scale = .01

values = .99*max_val*torch.ones((res, res, res),dtype=torch.float32, device='cuda')
cost = cost_scale*torch.ones((res,res, res),dtype=torch.float32, device='cuda')
boundary_conditions = torch.zeros((res,res, res),dtype=torch.float32, device='cuda')
boundary_types = torch.zeros((res,res, res),dtype=torch.float32, device='cuda')
# goal
goal_ind1 = int(res*15/30)
goal_ind2 = int(res*15/30)
goal_ind3 = int(res*15/30)
boundary_conditions[goal_ind1, goal_ind2, goal_ind3] = 0.0
boundary_types[goal_ind1, goal_ind2, goal_ind3] = 1.0



In [2]:
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim
from laplacian_py.network import LaplaceNet, compute_loss, calculate_gradient
    
network = LaplaceNet((res, res, res), max_val, 1*cost_scale, 0)
# network.cuda(device='cuda')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
network = network.to(device)

V_in = values.clone()

print(network.indexes)
print(network.C.shape)
print(network.num_dims)

tensor([-2551,   -51,  2449, -2501,    -1,  2499, -2451,    49,  2549, -2550,
          -50,  2450, -2500,  2500, -2450,    50,  2550, -2549,   -49,  2451,
        -2499,     1,  2501, -2449,    51,  2551], device='cuda:0',
       dtype=torch.int32)
torch.Size([50, 50, 50])
3


In [3]:
# grid_train_tmp = grid_train
# from torch.profiler import profile, record_function, ProfilerActivity
# # with profile(activities=[torch.profiler.ProfilerActivity.CUDA], record_shapes=True) as prof:
# with torch.autograd.profiler.profile(use_cuda=True) as prof:
#     with record_function("model_train"):
#         for i in range(1000):
#             pred, C, _ = network(grid_train, boundary_types, boundary_conditions)
#             grid_train_tmp = grid_train_tmp+1.0
# # print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
# print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
# prof.export_chrome_trace("trace_cuda.json")
# # assert(False) 

In [4]:
lr = 1e-2 # norm
# lr = 1e-4 # abs
momentum = 0.99
optimizer = optim.SGD(network.parameters(), lr=lr, momentum=momentum)

XYZ_train = [np.linspace(-1, 1, res, dtype=np.float32) for _ in range(3)]
X_train, Y_train, Z_train = np.meshgrid(*XYZ_train)
out = np.stack([X_train, Y_train, Z_train], axis=3)
grid_train = torch.tensor(out, dtype=torch.float32, device='cuda', requires_grad=True)
grid_train = grid_train.unsqueeze(axis=0)


# target = torch.tensor(5*X_train+5*Y_train +5*Z_train + 15,dtype=torch.float32, device='cuda')
# target = torch.tensor(15*Z_train + 15,dtype=torch.float32, device='cuda')
# target = torch.tensor(30*np.sqrt(X_train**2+Y_train**2),dtype=torch.float32, device='cuda')
target = torch.tensor(20*np.sqrt(X_train**2+Y_train**2+Z_train**2),dtype=torch.float32, device='cuda')

# target = torch.tensor(20*X + 0,dtype=torch.float32, device='cuda')
# target = torch.tensor(12*(X**2+Y**2+Z**2) + 0,dtype=torch.float32, device='cuda')
# target = torch.tensor(15 + 0*X,dtype=torch.float32, device='cuda')
# target = torch.tensor(10*Y + 10*(1-X), dtype=torch.float32, device='cuda')
# target = torch.tensor(10*(1-Y) + 10*X, dtype=torch.float32, device='cuda')


# target = target.unsqueeze(axis=0)
       
        
network.reset()
for it in range(0, 2000):
    optimizer.zero_grad()
    pred, C, _ = network(grid_train, boundary_types, boundary_conditions)
    loss_train = 1*compute_loss(pred=pred, target=target, C=C, cost_scale=cost_scale)
    if it % 10 == 0:
        print(loss_train)
    loss_train.backward()
    optimizer.step()
    




tensor(3629.4932, device='cuda:0', grad_fn=<MulBackward0>)
tensor(646.5619, device='cuda:0', grad_fn=<MulBackward0>)
tensor(616.9588, device='cuda:0', grad_fn=<MulBackward0>)
tensor(587.3112, device='cuda:0', grad_fn=<MulBackward0>)
tensor(556.6747, device='cuda:0', grad_fn=<MulBackward0>)
tensor(525.7611, device='cuda:0', grad_fn=<MulBackward0>)
tensor(498.5719, device='cuda:0', grad_fn=<MulBackward0>)
tensor(481.9961, device='cuda:0', grad_fn=<MulBackward0>)
tensor(474.6460, device='cuda:0', grad_fn=<MulBackward0>)
tensor(469.1779, device='cuda:0', grad_fn=<MulBackward0>)
tensor(463.9139, device='cuda:0', grad_fn=<MulBackward0>)
tensor(458.8194, device='cuda:0', grad_fn=<MulBackward0>)
tensor(453.9053, device='cuda:0', grad_fn=<MulBackward0>)
tensor(449.1833, device='cuda:0', grad_fn=<MulBackward0>)
tensor(444.6574, device='cuda:0', grad_fn=<MulBackward0>)
tensor(440.3277, device='cuda:0', grad_fn=<MulBackward0>)
tensor(436.1909, device='cuda:0', grad_fn=<MulBackward0>)
tensor(432.24

tensor(343.0538, device='cuda:0', grad_fn=<MulBackward0>)
tensor(343.0271, device='cuda:0', grad_fn=<MulBackward0>)
tensor(343.0017, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.9777, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.9550, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.9335, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.9133, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.8942, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.8763, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.8594, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.8437, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.8289, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.8152, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.8024, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.7906, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.7796, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.7696, device='cuda:0', grad_fn=<MulBackward0>)
tensor(342.760

In [5]:
torch.max(network.V)
max_val

80

In [30]:
boundary_types_2 = boundary_types.clone()
boundary_conditions_2 = boundary_conditions.clone()


# obj1_ind1 = int(res*18/30)
# obj1_ind2 = int(res*2/3)
# obj1_ind3 = int(res*0)
# obj1_ind4 = int(res*2/3)
# boundary_conditions_2[obj1_ind1:obj1_ind2, obj1_ind3:obj1_ind4] = max_val*10
# boundary_types_2[obj1_ind1:obj1_ind2, obj1_ind3:obj1_ind4] = 1.0

obj2_ind1 = int(res*8/30)
obj2_ind2 = int(res*15/30)
obj2_ind3 = int(res*5/30)
obj2_ind4 = int(res*25/30)
boundary_conditions_2[obj2_ind1:obj2_ind2, obj2_ind1:obj2_ind2, obj2_ind3:obj2_ind4] = max_val*10
boundary_types_2[obj2_ind1:obj2_ind2, obj2_ind1:obj2_ind2, obj2_ind3:obj2_ind4] = 1.0


# obj1_ind1 = int(res*1/3)
# obj1_ind2 = int(res*2/3)
# obj1_ind3 = int(res*1/3)
# obj1_ind4 = int(res*2/3)
# boundary_conditions_2[obj1_ind1:obj1_ind2, obj1_ind3:obj1_ind4] = max_val*10
# boundary_types_2[obj1_ind1:obj1_ind2, obj1_ind3:obj1_ind4] = 1.0


# tmp_inds = boundary_types_2 > 0
# tmp_inds = tmp_inds*(torch.rand(tmp_inds.shape,dtype=torch.float32, device='cuda')>.03)
# boundary_types_2[tmp_inds] = 0
# boundary_conditions_2[tmp_inds] = 0
# boundary_conditions_2[goal_ind1, goal_ind2] = 0.0
# boundary_types_2[goal_ind1, goal_ind2] = 1.0

network.reset()
for i in range(1000):
    pred, C_pos, J2 = network(grid_train, boundary_types_2, boundary_conditions_2)
    V_in = pred.detach()
 





In [31]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
import matplotlib
matplotlib.use('TkAgg')

# z = pred.cpu().detach().numpy()[1:-1, 1:-1, 1:-1]
# X = np.linspace(-15, 15, z.shape[0])
# Y = np.linspace(-15, 15, z.shape[1])
# Z = np.linspace(-15, 15, z.shape[2])
# X, Y, Z = np.meshgrid(X, Y, Z)


# # ax = plt.figure().add_subplot(projection='3d')
# ax = plt.figure(figsize=(20, 20)).add_subplot(projection='3d')

# Up = -np.diff(z[1:-2, 2:-2, 2:-2], axis=0)
# Vp = -np.diff(z[2:-2, 1:-2, 2:-2], axis=1)
# Wp = -np.diff(z[2:-2, 2:-2, 1:-2], axis=2)

# ax.quiver(X[2:-2, 2:-2, 2:-2], Y[2:-2,2:-2,2:-2], Z[2:-2,2:-2,2:-2], Vp, Up, Wp, length=0.5, normalize=True)

# ax.view_init(30, 30, 0)
# # f = plt.figure(figsize=(20, 20))
# plt.show()
# plt.pause(0)

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/home/paul/CLionProjects/thesis_nerf/venv/lib/python3.10/site-packages/matplotlib/backends/_backend_tk.py", line 552, in destroy
    Gcf.destroy(self)
  File "/home/paul/CLionProjects/thesis_nerf/venv/lib/python3.10/site-packages/matplotlib/_pylab_helpers.py", line 66, in destroy
    manager.destroy()
  File "/home/paul/CLionProjects/thesis_nerf/venv/lib/python3.10/site-packages/matplotlib/backends/_backend_tk.py", line 587, in destroy
    delayed_destroy()
  File "/home/paul/CLionProjects/thesis_nerf/venv/lib/python3.10/site-packages/matplotlib/backends/_backend_tk.py", line 577, in delayed_destroy
    self.window.destroy()
  File "/usr/lib/python3.10/tkinter/__init__.py", line 2341, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed
Ex

TclError: can't invoke "destroy" command: application has been destroyed

In [37]:
# old way
dt = 0.01
ax = plt.figure().add_subplot(projection='3d')

grid_pred = pred[1:-1, 1:-1, 1:-1].unsqueeze(axis=0).unsqueeze(axis=0)

for x_val in [-.95, -.5, 0, .5 ,.95]:
    for y_val in [-.95, -.5, 0, .5 ,.95]:
        for z_val in [-.95, 0.0, .95]:
            cur = np.reshape(np.array([x_val, y_val, z_val]),(1, 1, 1, 1, 3))
            traj = np.zeros((1000, 3))
            for i in range(traj.shape[0]):
                query = torch.tensor(cur, dtype=torch.float32, device='cuda', requires_grad=True)
                query.unsqueeze(axis=0)
                J1, J2 = calculate_gradient(grid_pred, query)
                J2 = J2.cpu().detach().numpy()
                J2 = J2/(np.linalg.norm(J2) + .00000000001)
                cur += -J2*dt
#                 print(J2)
#                 print(cur)
                traj[i, :] = cur.squeeze()

            ax.plot(traj[:,0], traj[:,1], traj[:,2])
    
obj_inds = (boundary_types_2 == 1).cpu().numpy()
ax.scatter(X_train[obj_inds], Z_train[obj_inds],  Y_train[obj_inds])    
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()
plt.pause(0)


In [36]:
# dt = 0.01
# ax = plt.figure().add_subplot(projection='3d')

# for x_val in [-.99, -.5, 0, .5 ,.99]:
#     for y_val in [-.99, -.5, 0, .5 ,.99]:
#         for z_val in [-.99]:
#             cur = np.reshape(np.array([x_val, y_val, z_val]),(1, 1, 1, 1, 3))
#             traj = np.zeros((1000, 3))
#             for i in range(traj.shape[0]):
#                 query = torch.tensor(cur, dtype=torch.float32, device='cuda', requires_grad=True)
#                 query.unsqueeze(axis=0)
#                 _, _, J2 = network(query, boundary_types_2, boundary_conditions_2, calc_gradient=True)
#                 J2 = J2.cpu().detach().numpy()
#                 J2 = J2/(np.linalg.norm(J2) + .00000000001)
#                 cur += -J2*dt
#                 traj[i, :] = cur     

#             ax.plot(traj[:,0], traj[:,1], traj[:,2])

# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')
# plt.show()

# plt.pause(0)

In [24]:
network.V[2,2,2]
# boundary_types_2[2,2,2]

tensor(34.2956, device='cuda:0')

In [25]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
                               
val = pred[1:-1, 1:-1, 1:-1].cpu().detach().numpy()

skip = 2
color = val[0:-1:skip, 0:-1:skip, 0:-1:skip]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(X[0:-1:skip,0:-1:skip,0:-1:skip], Y[0:-1:skip,0:-1:skip,0:-1:skip], Z[0:-1:skip,0:-1:skip,0:-1:skip],
          c=color)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()
plt.pause(0)

NameError: name 'X' is not defined

In [ ]:
X[0:-1:skip,0:-1:skip,0:-1:skip].shape
color.shape
X.shape
val.shape

In [ ]:
import itertools
res_in = (res, res, res)
num_dims = len(res_in)
dims_linear = torch.zeros(num_dims, dtype=torch.int32)
base = 1
for ind, d in enumerate(res_in):
    dims_linear[ind] = base
    base = base * d

indexes = torch.tensor([torch.sum(torch.tensor(v, dtype=torch.int32) * dims_linear) for v in
                     list(itertools.product([-1, 1], repeat=num_dims)) if
                     not all(vi == 0 for vi in v)], dtype=torch.int32, device='cuda')

indexes